In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import HTML
%matplotlib inline

<img src="astropy_banner_96.png"/>

# AstroPy: Basic Astronomy tools in Python
Karl Kosack (SAp, LEPCHE)

## What is AstroPy?

WebSite and Documentation : http://www.astropy.org

Previously there were a number of astronomy-realted packages that were scattered throughout the Python world. Generally each had a specific use, and they were not interoperable. 

<span style="color: darkcyan">Recently, AstroPy changed all that! now there is a **central package with all astronomy-related modules**! It collects the best of what was available, and provides a unified interface. </span>

In [ ]:
HTML('<iframe src="http://docs.astropy.org/en/stable/" width=800 height=500></iframe>')

### What we will cover:

* Units and Constants
* Coordinates Systems
    * spatial 
    * time
* Astronomical Table Access
* Astronomical Image Access

## Units and Constants

Very useful when doing computations in Python! Units are **convertable**, **combinable**,  and **propegate automatically**!

In [ ]:
from astropy import units as u

Defining a quanity that has a unit:

In [ ]:
d = 12.3 * u.kpc
M = np.linspace( 50,60,10 ) * u.kg

print(d)
print(M)

Note that units are associated with a scalar or an NDArray, and are thus efficient (only one per n-dimensional vector)

Now let's see the power of units:

### Simple Unit Conversion

In [ ]:
print( d.to( u.m ) )
print( d.to( u.Angstrom ) )
print( d.to( u.imperial.foot ) )
print( d.to( u.lightyear ) )

# can also use a string representation:
print(d.to( "m" ))

In [ ]:
print( d.to( u.kg ) )   # this fails of course! 

Getting at the value directly: use the ``value`` property (useful if you need to pass something that has units into a function that doesn't understand units, like for plotting)

In [ ]:
print("Distance in feet=", d.to( u.imperial.foot).value)   #strips off any units

### Composing units

In [ ]:
dNdE = 1e-10 * u.TeV/u.cm**2/u.s
print(dNdE)

In [ ]:
dNdE   # the IPython Notebook knows how to do it in a nicer way! 

You can even convert between complex units:

In [ ]:
print(dNdE.to( u.erg/u.m**2 * u.Hz))
print(dNdE.to( "erg cm^-2 s^-1"))

get the unit string in LaTeX:

In [ ]:
print(dNdE.unit.to_string(format='latex'))
dNdE.unit   # in notebook, automatically will "pretty print"

Lots of astonomy-compatible string representations!

In [ ]:
print(dNdE.unit.to_string( format='vounit' ))  #virtual-observatory complient
print(dNdE.unit.to_string( format='fits' ))    # FITS complient
print(dNdE.unit.to_string( format='cds' ))     #CDS Strasbourg table complient
print(dNdE.unit.to_string( format='console' )) # console pretty-print

#### Other interesting things you can do:

In [ ]:
print(dNdE.cgs)
print(dNdE.si)
print(u.Newton.decompose())

In [ ]:
u.kg.find_equivalent_units()

In [ ]:
# How much do I weigh in Solar Masses?
(85*u.kg).to( u.M_sun )

There are also many **CONSTANTS** that are defined in ``astropy.constants``.  Constants are  Quantities that have units units, and also have some extra metadata (a bibliographical reference, and error)

In [ ]:
nu = np.logspace(1,5,100) * u.Hz
Y = 100*(nu/1*u.Hz)**(-2.0)

plt.loglog( nu.value, Y.value)
plt.xlabel("Frequency (Hz)")

Let's convert this now from frequency (Hz) to energy (erg)! 

In [ ]:
E = nu.to( u.erg )

Of course, that fails, since there is a factor of planck's constant that we need to apply... We could do it using astropy.constants.h, but there's an even better way: **units can have some physics knowledge!**.  This is expressed by giving an *equivalencies* option:

In [ ]:
E = nu.to( u.eV, equivalencies=u.spectral() )
print(E.unit)

In [ ]:
plt.loglog( E.value, Y.value )
plt.xlabel( "eV" )

In [ ]:
print(dir(u.equivalencies))


-----------------------------------------

## Coordinates and Times
Some common task in astronomy are:
* convert positions between spatial coordinate systems
* lookup the coordinates of an object by name
* work with times, and convert them between time representations

In [ ]:
from astropy import coordinates as c
from astropy import time as t

First, let's get a coordinate object and play with it! (let's start with the cool stuff:)

In [ ]:
crabpos = c.SkyCoord.from_name( "Crab Nebula" ) 
print(crabpos)
print("--------")
print(crabpos.ra)
print(crabpos.ra.deg)
print(crabpos.ra.hour)

Some cool features of SkyCoords:

In [ ]:
etacarpos = c.SkyCoord.from_name("Eta Car")
print("Distance from Eta Car to the Crab is: ", etacarpos.separation( crabpos ))

In [ ]:
print(c.SkyCoord("12h16m35s", "-10d45m19s")) #other string formats are automaticaly understood

# or we can use values directly:
pos1 = c.SkyCoord( 10*u.hour, -20*u.deg )   
print(pos1)

# let's make one in another frame (default was ICRS, but let's use FK4, i.e. "B1950")
pos2 = c.SkyCoord( 10*u.hour, -20*u.deg, frame=c.FK4)
print(pos2)

In [ ]:
# note that RA/Dec expressed ICRS and FK4 are not the same! 
# What is the angualr distance between our two coordinates?
pos1.separation(pos2)

In [ ]:
print("sep in deg:", pos1.separation(pos2).deg)
print("sep in rad:", pos1.separation(pos2).rad)

Let's do some conversion: just use the appriate property:
* icrs
* fk5
* fk4
* galactic
* galactocentric
* altaz (see later...)

In [ ]:
print("ICRS:", pos1.icrs)
print("GALACTIC:", pos1.galactic)  # common helper attribute
print("GALACTIC:", pos1.transform_to( c.Galactic ))  # more general: transform to a frame
print("")
print("")
print("The Crab Nebula is {0:.1f} deg off-plane"\
    .format(crabpos.galactic.b.deg))

### More complex conversion: go from RA/Dec to Alt/Az! 

In [ ]:
crab = c.SkyCoord.from_name("Crab Nebula")

# need to know two pieces of info: Location on Earth + time!
paris = c.EarthLocation( lat=48.8567*u.deg, lon=2.3508*u.deg )
print(paris)

now = t.Time.now()  # gets the current time
print(now)

crab_altaz = crab.transform_to( c.AltAz(obstime=now, location=paris) )

print("CRAB IS AT ELEVATION: {0:.3f} deg , AZIMUTH: {1:.3f} deg"\
    .format( crab_altaz.alt.deg, crab_altaz.az.deg))

> NOTE: you can even give an atmospheric pressure to the AltAz frame to get even more accuracy with refraction corrections!

### Time Conversion
Times are similar to SkyCoordinates, so we won't go into as much detail.  



-----------------------------------

## Table Access

One of the most powerful tools in AstroPy is the table access library.  It allows one to work with tables in many different formats with ease! You don't even normally have to care what the format is.

### reading tables

In [ ]:
from astropy import table

# let's open an existing FITS table:

atnf = table.Table.read( "atnf_PSR_07.fits" )


In [ ]:
atnf

In [ ]:
print("COLUMNS:",atnf.colnames)
print("")
print("HEADERS:")
for key in atnf.meta:   # headers are key=value in an OrderedDict
    print("    -> ",key,"=",atnf.meta[key])

In [ ]:
atnf['NAME']

In [ ]:
# make a smaller table from this table, with only the cols you want:
small = atnf['NAME','RA_OBJ','DEC_OBJ']
small

The display in the Notebook is nice, but we can also do it fancier:

In [ ]:
atnf.show_in_browser( jsviewer=True )

> Note that it is just concidence that we loaded a FITS table. We will see later we can load   >  tables in nearly any format! ASCII (multiple formats), HDF5, HTML, LATEX, etc. Even very  >astro-specific formats like **ascii.sextractor**.

###Tables and Columns act like NumPy NDArrays! 

In [ ]:
print(atnf['EDOTD2'][10:15]) # just entries 10 to 15
print(atnf[10:15]) # just entries 10 to 15

In [ ]:
# you can even get at just the data exactly as NumPy array:
atnf['EDOTD2'].data

so you can use the same slicing features of NumPy to make selections of your data!

In [ ]:
# find just the sources that have a spin-down flux above 1e36:
selected = atnf[ atnf['EDOTD2'] > 1e36 ]
selected

In [ ]:
print("The following {0} are very powerful pulsars!".format( len(selected) ))
print(selected['NAME'])

### writing tables
Let's now write the selected pulsars to **another FITS file**!

In [ ]:
selected.write("selected.fits")
#that's it!

#DEMO: open that fits file in a terminal...

But that's not all: we have **many formats** to choose from:

http://docs.astropy.org/en/stable/io/unified.html#built-in-table-readers-writers


In [ ]:
selected.write("selected.tex", format="latex") # publication ready!
selected.write("snrs.h5", "/SNRs", format="hdf5") # some need extra arguments

! cat selected.tex


note that you can **read tables from URLs** too:
    

In [ ]:
snrs = table.Table.read("ftp://cdsarc.u-strasbg.fr/pub/cats/VII/253/snrs.dat",
                         readme="ftp://cdsarc.u-strasbg.fr/pub/cats/VII/253/ReadMe",
                         format="ascii.cds")

print("COLS: ",snrs.colnames)
snrs[:5]

### Making a table from scratch 

We don't always want to load a table to work with it.. SOmetimes we want to generate some data and write it to a table

In [ ]:
a = [1, 4, 5]
b = [2.0, 5.0, 8.2]
c = ['x', 'y', 'z']
t = table.Table([a, b, c], names=('a', 'b', 'c'), meta={'name': 'first table'})
print(t)
t.meta  # header keywords

In [ ]:
table.Table?

### Cool stuff: that I don't have time to show you
* joining tables
* fancy masking tables
* stacking tables



---------------------------


## FITS Image access

Ok, we can access FITS tables (and other kinds as well), but what about FITS images and datacubes?  For taht we need to go to the lower-level fits interface.  

SOme of you make have used ``pyfits`` in the past. Development been moved ``astropy.io.fits`` (pyfits is not longer supported)

In [ ]:
from astropy.io import fits


In [ ]:
fitsfile = fits.open("vela_2.0-8.0_flux.fits")

fitsfile.info()


In [ ]:
image = fitsfile[0] # first HDU (there's only one in this file, but may be more)

In [ ]:
image.header

get a header keyword:

In [ ]:
print(image.header['MJD_OBS'])

In [ ]:
image.data

It's just a **NumPy** ``NDArray`` again!  Easy to work with...

In [ ]:
plt.imshow( image.data, origin='lower' )
plt.colorbar()

In [ ]:
print("Value Range: ",image.data.min(), image.data.max())

Let's scale the plot nicer by taking the SQRT and changing the min/max values

In [ ]:
obs = t.Time( "2015-10-11 15:17:45.3", format="iso", scale="utc")
print(obs)
print(obs.mjd)  # Modified Julian Day
print(obs.gps)  # GPS time
print(obs.unix) # UNIX timestamp
print(obs.iso)  # ISO string
print(obs.isot) # ISO string with T
print(obs.tt)   # Terrestrial Time
print(obs.tai)  # International atomic time
print(obs.datetime) # python datetime object
# etc...

How about differences?

In [ ]:
plt.imshow( np.sqrt(image.data), vmin=0, vmax=np.sqrt(0.3e-5), origin='lower' )
plt.colorbar()

In [ ]:
clip = image.data[100:300,100:300]


####Basic Image Smoothing

In [ ]:
from astropy import convolution as conv
from astropy import visualization as vis
from astropy.visualization.mpl_normalize import ImageNormalize

# gaussian-smooth the image with a std of 8.0 pixelsl
gauss = conv.Gaussian2DKernel( 8.0 )
smooth = conv.convolve_fft( clip, gauss ) 
plt.imshow( np.sqrt(smooth), vmin=0, vmax=np.sqrt(0.3e-5), origin='lower' )

#### Fancier image scaling/plotting (like DS9)

In [ ]:
stretch =  vis.ContrastBiasStretch(0.2,0.9)  # there are many other types
interval = vis.PercentileInterval(95.0)   # many other types too

limits = interval.get_limits( image.data )
norm = ImageNormalize( vmin=limits[0], vmax=limits[1], 
                        stretch=stretch, clip=False )

fig, ax = plt.subplots(1,2)
im1 = ax[0].imshow( clip, norm=norm, origin='lower' )
im2 = ax[1].imshow( smooth, norm=norm, origin='lower' )

In [ ]:
R = fits.open( "casa_0.5-1.5keV.fits" )[0].data 
G = fits.open( "casa_1.5-3.0keV.fits" )[0].data
B = fits.open( "casa_4.0-6.0keV.fits" )[0].data

plt.hot()
plt.figure( figsize=(13,3) )
plt.subplot( 1,3,1 )
plt.imshow( R, origin='lower', vmax=10 )
plt.subplot( 1,3,2 )
plt.imshow( G, origin='lower', vmax=10)
plt.subplot( 1,3,3 )
plt.imshow( B, origin='lower',vmax=10 )


In [ ]:
stacked = np.array( (R/R.max(),G/G.max(),B/B.max()) ).T 
print(stacked.shape)

**note:**
  - we took the Transpose to make it 1024 x 1024 x 3 , rather than 3 x 1024 x 1024
  - we did some division since RGB images must be in the range (0,1) for each plane 
  
Now, let's plot it! **imshow** understands RGB images if they are a NxMx3 NDArray

In [ ]:
t0 = t.Time( "2016-01-01" ) # defaults to ISO UTC
t1 = t.Time.now()

diff = t1-t0
diff

In [ ]:
print("time since {0} is {1} seconds".format( t0, diff.sec))

In [ ]:
# Generally it's good to work with vectors of times, since it's faster.
# let's make a time vector and add some times

t0 = t.Time('1999-01-01T00:00:00.123456789')
dt = t.TimeDelta( 1*u.day )
deltas = dt * np.linspace(0.,1.,12)

times = t0 + deltas
print(times.iso)

In [ ]:
t0 = t.Time('1999-01-01T00:00:00.123456789')
dt = t.TimeDelta( 1*u.minute )
deltas = dt * np.linspace(0.,1500,100)
times = t0 + deltas

crab_altaz = crab.transform_to( c.AltAz(obstime=times, location=paris) )

plt.plot( times.jd, crab_altaz.alt )
plt.ylabel("Altitude of Crab")


In [ ]:
from astropy import constants

In [ ]:
print( constants.M_sun )

In [ ]:
print(constants.R_jup)

In [ ]:
print("Jupiter's radius is",constants.R_jup.to( u.km ))
print("Jupiter is",constants.R_jup/constants.R_earth," x the radius of earth")

### Advanced unit conversion: